<a href="https://colab.research.google.com/github/Engr-Genius/DATA-ANALYSIS-WITH-PYTHON/blob/main/Python%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
import numpy as np

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!ls /content/drive

/content/drive/MyDrive/job_change_data.csv


In [24]:
df = pd.read_csv('/content/drive/MyDrive/job_change_data.csv')
df.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,Oct-49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


Filter rows where City_Development_Index > 0.8 and Company_Size > 3

In [32]:
import numpy as np

def convert_company_size(size_str):
    if pd.isna(size_str):
        return np.nan
    s = str(size_str).strip()
    if '-' in s:
        # Handle 'Oct-49' as a specific case, assuming 'Oct' is a typo for '10'
        if s.lower() == 'oct-49':
            return 10
        try:
            return int(s.split('-')[0]) # e.g., '10-49' becomes 10
        except ValueError:
            return np.nan # If the first part is not a valid integer
    elif '<' in s:
        return 1 # e.g., '<10' becomes 1 (as lowest possible size)
    elif '+' in s:
        try:
            return int(s.replace('+', '')) # e.g., '10000+' becomes 10000
        except ValueError:
            return np.nan
    else:
        try:
            return int(s)
        except ValueError:
            return np.nan # For any other unparseable strings

df['company_size_numeric'] = df['company_size'].apply(convert_company_size)

filtered_df = df[(df['city_development_index'] > 0.8) &
                 (df['company_size_numeric'] > 3)]

Use iloc to select first 10 rows and specific columns

In [34]:
subset_df = df.iloc[:10][['experience', 'education_level']]

Group by Relevant_Experience and compute average CDI

In [36]:
avg_cdi = df.groupby('relevent_experience')['city_development_index'].mean()

Group by Company_Size and count unique entries in Last_New_Job

In [38]:
unique_last_new_job = df.groupby('company_size')['last_new_job'].nunique()

Frequency distribution of Company_Type

In [40]:
company_type_freq = df['company_type'].value_counts()

Identify numerical columns with missing values and fill with mean

In [41]:

num_cols = df.select_dtypes(include='number')

# columns with missing values
missing_num_cols = num_cols.columns[num_cols.isnull().any()]

# fill with mean
df[missing_num_cols] = df[missing_num_cols].fillna(df[missing_num_cols].mean())

Drop rows where >50% of data is missing + analyze impact

In [42]:

initial_rows = df.shape[0]

df_cleaned = df.dropna(thresh=df.shape[1] * 0.5)

final_rows = df_cleaned.shape[0]
impact = initial_rows - final_rows

print("Rows removed:", impact)

Rows removed: 0


Query rows where Experience > 10 and Company_Size == 7

In [45]:
import numpy as np

def convert_experience(exp_str):
    if pd.isna(exp_str):
        return np.nan
    s = str(exp_str).strip()
    if s == '<1':
        return 0
    elif s == '>20':
        return 21 # Representing 'more than 20 years'
    else:
        try:
            return int(s)
        except ValueError:
            return np.nan # For any other unparseable strings

df['experience_numeric'] = df['experience'].apply(convert_experience)

query_df = df[(df['exwperience_numeric'] > 10) & (df['company_size_numeric'] == 7)]

Create new feature Experience_Gap

In [48]:
import numpy as np

def convert_last_new_job(job_str):
    if pd.isna(job_str):
        return np.nan
    s = str(job_str).strip()
    if s == 'never':
        return 0
    elif s == '>4':
        return 5 # Representing 'more than 4 years'
    else:
        try:
            return int(s)
        except ValueError:
            return np.nan

df['last_new_job_numeric'] = df['last_new_job'].apply(convert_last_new_job)
df['Experience_Gap'] = df['experience_numeric'] - df['last_new_job_numeric']

Normalize CDI to 0–1 scale

In [50]:
df['CDI_Normalized'] = (df['city_development_index'] -
                        df['city_development_index'].min()) / \
                       (df['city_development_index'].max() -
                        df['city_development_index'].min())

Create cdi_per column, merge, and analyze

In [52]:

# Create new column

df_new = pd.DataFrame()
df_new['cdi_per'] = df['city_development_index'] * 100
df_new['id'] = df.index  # key for merging

# Add ID to original DF

df['id'] = df.index

# Merge

merged_df = pd.merge(df, df_new, on='id', how='left')

# Insights you can analyze:

merged_df[['city_development_index', 'cdi_per']].head()

,city_development_index,cdi_per
0,0.827,82.7
1,0.920,92.0
2,0.624,62.4
3,0.827,82.7
4,0.920,92.0
